In [1]:
import os
from tensorflow.keras.models import load_model

# Директории моделей
PATH_TO_MODELS = r'../../models' 

model_name = f'model_Sequential'
model_path = os.path.join(PATH_TO_MODELS, model_name + '.h5')

model = load_model(filepath=model_path, compile=True)

In [2]:
# encoder_features = [feature for feature in columns_for_encoder if feature in modeled_columns]
# scaler_features = [feature for feature in columns_for_scaler if feature in modeled_columns]

dictionary = dict(
    target_column = 'Churn',
    encoder_features = [
        'SeniorCitizen',
        'Dependents',
        'PhoneService',
        'MultipleLines',
        'InternetService',
        'OnlineSecurity',
        'OnlineBackup',
        'DeviceProtection',
        'TechSupport',
        'Contract'],
    scaler_features = [
        'SeniorCitizen',
        'Dependents',
        'tenure',
        'PhoneService',
        'MultipleLines',
        'InternetService',
        'OnlineSecurity',
        'OnlineBackup',
        'DeviceProtection',
        'TechSupport',
        'Contract',
        'MonthlyCharges']
)

dictionary_name = 'dictionary'
dictionary_path = os.path.join(PATH_TO_MODELS, dictionary_name + '.json')

with open(dictionary_path, 'w') as fid:
    json.dump(dictionary, fid)

In [3]:
with open(dictionary_path, 'rb') as fid:
    q = json.load(fid)

Setting environment variables

In [7]:
%env MLFLOW_S3_ENDPOINT_URL=https://your_S3_endpoint_url/
%env AWS_ACCESS_KEY_ID=your_S3_access_key_id
%env AWS_SECRET_ACCESS_KEY=your_S3_secret_key

env: MLFLOW_S3_ENDPOINT_URL=https://your_S3_endpoint_url/
env: AWS_ACCESS_KEY_ID=your_S3_access_key_id
env: AWS_SECRET_ACCESS_KEY=your_S3_secret_key


In [5]:
BUCKET = 'your_bucket'
ARTIFACT_STORE = 'your_artifact_store'

artifact_location=f's3://{BUCKET}/{ARTIFACT_STORE}'

experiment_name = 'model_churn'
mlflow_uri = "http://localhost:5000"
model_name='tf-Sequential'

In [6]:
import mlflow
from mlflow.models.signature import infer_signature



mlflow.set_tracking_uri(mlflow_uri)


try:
    experiment = mlflow.get_experiment_by_name(
        name=experiment_name,
    )
    experiment_id = experiment.experiment_id
except AttributeError:
    experiment_id = mlflow.create_experiment(
        name=experiment_name, 
        artifact_location=artifact_location,
    )

with mlflow.start_run(
    experiment_id=experiment_id
) as run:
    
    # log model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=experiment_name,
        registered_model_name=model_name,
    )
    
    # log dictionary
    mlflow.log_artifact(local_path=dictionary_path)
    
    # log targer label encoder
    feature = dictionary.get('target_column')
    label_encoder_name = f'label_encoder_{feature}'
    label_encoder_path = os.path.join(PATH_TO_MODELS, label_encoder_name + '.pkl')
    mlflow.log_artifact(local_path=label_encoder_path,)    
    
    # log feature encoders
    for feature in dictionary.get('encoder_features'):
        label_encoder_name = f'label_encoder_{feature}'
        label_encoder_path = os.path.join(PATH_TO_MODELS, label_encoder_name + '.pkl')
        mlflow.log_artifact(local_path=label_encoder_path,)
    
    # log feature scalers
    for feature in dictionary.get('scaler_features'):
        feature_scaler_name = f'feature_scaler_{feature}'
        feature_scaler_path = os.path.join(PATH_TO_MODELS, feature_scaler_name + '.pkl')
        mlflow.log_artifact(local_path=feature_scaler_path,) 
    
    mlflow.end_run()

INFO:tensorflow:Assets written to: ram://72c9de48-c05b-49ae-a708-caf8680bd867/assets


2022/05/08 18:52:51 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpeheq1175/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==0.24.2', 'cloudpickle==1.6.0']. Set logging level to DEBUG to see the full traceback.
Registered model 'tf-Sequential' already exists. Creating a new version of this model...
2022/05/08 18:52:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: tf-Sequential, version 10
Created version '10' of model 'tf-Sequential'.
